<a href="https://colab.research.google.com/github/TuliDas/Healthcare-Customer-Support-ChatBot/blob/main/healthcare__chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Tools**
Note: For text generation, alwyas use T5

In [1]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments


In [3]:
df = pd.read_csv("/content/domain_specific_chatbot_data.csv")
df.head()

,query,response,intent,domain
0,What are the side effects of the COVID-19 vacc...,Common side effects of the COVID-19 vaccine in...,side effects inquiry,healthcare
1,How can I schedule an appointment with my doctor?,You can schedule an appointment by calling our...,appointment booking,healthcare
2,What should I do if I miss a dose of my medica...,"If you miss a dose, take it as soon as you rem...",medication inquiry,healthcare
3,How can I check my account balance?,You can check your balance by logging into you...,balance inquiry,finance
4,What is the interest rate for a personal loan?,The current interest rate for personal loans i...,loan inquiry,finance


**Data Preprocessing**

In [4]:
df['query'][0]

'What are the side effects of the COVID-19 vaccine?'

In [5]:
df['response'][0]

'Common side effects of the COVID-19 vaccine include soreness at the injection site, fever, and fatigue.'

In [6]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2) # 80% data on train set, 20% test set
train_df.shape, val_df.shape

((2400, 4), (600, 4))

In [7]:
train_df
# indexing are not starting from zero

,query,response,intent,domain
2392,What should I do if I miss a dose of my medica...,"If you miss a dose, take it as soon as you rem...",medication inquiry,healthcare
377,How do I update my contact details on my account?,"To update your contact details, log into your ...",contact update,finance
1135,"I lost my credit card, what should I do?",Please contact our customer service immediatel...,lost card reporting,finance
2617,How do I update my contact details on my account?,"To update your contact details, log into your ...",contact update,finance
2017,How do I update my contact details on my account?,"To update your contact details, log into your ...",contact update,finance
...,...,...,...,...
1499,How do I apply for a student loan?,You can apply for a student loan by visiting o...,student loan application,finance
2498,Can I make changes to my loan repayment schedule?,Changes to your loan repayment schedule can be...,loan repayment adjustment,finance
2675,"I lost my credit card, what should I do?",Please contact our customer service immediatel...,lost card reporting,finance
2805,"I lost my credit card, what should I do?",Please contact our customer service immediatel...,lost card reporting,finance


In [8]:
# make indexing starts from zero

train_data = train_df.reset_index(drop=True)
validation_data = val_df.reset_index(drop=True)
validation_data

,query,response,intent,domain
0,What is the interest rate for a personal loan?,The current interest rate for personal loans i...,loan inquiry,finance
1,What are the side effects of the COVID-19 vacc...,Common side effects of the COVID-19 vaccine in...,side effects inquiry,healthcare
2,"I lost my credit card, what should I do?",Please contact our customer service immediatel...,lost card reporting,finance
3,How do I update my contact details on my account?,"To update your contact details, log into your ...",contact update,finance
4,What should I do if I miss a dose of my medica...,"If you miss a dose, take it as soon as you rem...",medication inquiry,healthcare
...,...,...,...,...
595,How can I check my account balance?,You can check your balance by logging into you...,balance inquiry,finance
596,How can I schedule an appointment with my doctor?,You can schedule an appointment by calling our...,appointment booking,healthcare
597,What is the interest rate for a personal loan?,The current interest rate for personal loans i...,loan inquiry,finance
598,What is the interest rate for a personal loan?,The current interest rate for personal loans i...,loan inquiry,finance


In [9]:
# cleaning text
import re

def clean_text(text):
    text = text.strip().lower()  # Strip and convert to lower case
    text = re.sub(r'\r\n', ' ', text)  # Remove carriage returns and line breaks
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'<.*?>', '', text)  # Remove any XML tags
    return text

# apply cleaning to the responses and query columns
train_data['query'] = train_data['query'].apply(clean_text)
train_data['response'] = train_data['response'].apply(clean_text)
validation_data['query'] = validation_data['query'].apply(clean_text)
validation_data['response'] = validation_data['response'].apply(clean_text)

# display a sample
validation_data

,query,response,intent,domain
0,what is the interest rate for a personal loan?,the current interest rate for personal loans i...,loan inquiry,finance
1,what are the side effects of the covid-19 vacc...,common side effects of the covid-19 vaccine in...,side effects inquiry,healthcare
2,"i lost my credit card, what should i do?",please contact our customer service immediatel...,lost card reporting,finance
3,how do i update my contact details on my account?,"to update your contact details, log into your ...",contact update,finance
4,what should i do if i miss a dose of my medica...,"if you miss a dose, take it as soon as you rem...",medication inquiry,healthcare
...,...,...,...,...
595,how can i check my account balance?,you can check your balance by logging into you...,balance inquiry,finance
596,how can i schedule an appointment with my doctor?,you can schedule an appointment by calling our...,appointment booking,healthcare
597,what is the interest rate for a personal loan?,the current interest rate for personal loans i...,loan inquiry,finance
598,what is the interest rate for a personal loan?,the current interest rate for personal loans i...,loan inquiry,finance


**TOKENIZATION**

In [10]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
# preprocessing function

def preprocess_function(examples):
    inputs = tokenizer(examples['query'], padding='max_length', truncation=True,  max_length=250)
    outputs = tokenizer(examples['response'], padding='max_length', truncation=True,  max_length=250)
    inputs['labels'] = outputs['input_ids']  # create a new key to 'Inputs' to track the output ids
    return inputs

# Apply the preprocessing
train_dataset = train_data.apply(preprocess_function, axis=1)
validation_dataset = validation_data.apply(preprocess_function, axis=1)

In [12]:
train_data['query'][0]

'what should i do if i miss a dose of my medication?'

In [13]:
# it is a dictonary
# input_ids are the query's tokenizations form
# labels are the response's tokenization form
train_dataset[0]

{'input_ids': [125, 225, 3, 23, 103, 3, 99, 3, 23, 3041, 3, 9, 6742, 13, 82, 7757, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

**Fine-Tuning the Model**

In [14]:

# Model call
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory for checkpoints
    num_train_epochs=6,              # number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="./logs",            # directory for storing logs
    logging_steps=50,                # how often to log training info
    save_steps=500,                  # how often to save a model checkpoint
    eval_steps=50,                   # how often to run evaluation
    evaluation_strategy="epoch",     # Ensure evaluation happens every `epoch`
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset
)

# Train the model
trainer.train()



model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.030100,0.006489
2,0.001500,0.000032
3,0.000600,0.000009
4,0.000400,0.000005
5,0.000300,0.000004
6,0.000300,0.000004


TrainOutput(global_step=1800, training_loss=0.6238630769168958, metrics={'train_runtime': 2051.5139, 'train_samples_per_second': 7.019, 'train_steps_per_second': 0.877, 'total_flos': 4281735168000000.0, 'train_loss': 0.6238630769168958, 'epoch': 6.0})

Save and Load Model

In [15]:
model.save_pretrained("./chatbot_model")
tokenizer.save_pretrained("./chatbot_model")

model = T5ForConditionalGeneration.from_pretrained("./chatbot_model")
tokenizer = T5Tokenizer.from_pretrained("./chatbot_model")

ChatBot System

In [16]:
device = model.device

def chatbot(query):
  # repeat all previous steps
  # cleaning the input query text
  query = clean_text(query)
  # tokenize the query
  input_ids = tokenizer(query, return_tensors="pt", truncation=True,  max_length=250)

  inputs = {key: value.to(device) for key, value in input_ids.items()} # inputs is a dictionary

  outputs = model.generate(
        input_ids["input_ids"],
        max_length=250,
        num_beams=5,
        early_stopping=True
  )
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

while True:
  user_input = input("You: ")
  if user_input.lower() == "exit":
    break
  response = chatbot(user_input)
  print("Chatbot:", response)

You: How to login to the system ?
Chatbot: you can login to the system by logging into the system or using our mobile app.
You: Where to find setting options?
Chatbot: you can find setting options by logging into your account online or using our mobile app.
You: What are the side effects of the COVID-19 vaccine?
Chatbot: common side effects of the covid-19 vaccine include soreness at the injection site, fever, and fatigue.
You: exit


Download Model to your PC

In [ ]:
import shutil

# Compress the saved model directory
shutil.make_archive("chatbot_model", 'zip', "./chatbot_model")


from google.colab import files

# Download the zip file to your PC
files.download("chatbot_model.zip")